In [50]:
# importing necessary packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import joblib


In [25]:
#load dataset
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/filtered_merged_data.csv')
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                Model          Brand  Year  Price  Mileage Transmission
0       Nissan Almera         Nissan  2015  21000   149490    Automatic
1        Perodua Aruz        Perodua  2019  50600    88806    Automatic
2       Daihatsu Gran       Daihatsu  2021  40000   104943       Manual
3  Mercedes-Benz C250  Mercedes-Benz  2010  43000   147362    Automatic
4         Proton Iriz         Proton  2019  31900    52965    Automatic


In [20]:
# Normalise numerical variables
scaler = StandardScaler()
df[['Mileage', 'Year']] = scaler.fit_transform(df[['Mileage', 'Year']])

In [21]:
# Train-test split
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Linear Regression Model
# Encoding categorical data
categorical_columns = ['Model', 'Brand', 'Transmission']

# One-hot encoding
encoder = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
], remainder='passthrough')

# Fit and transform X_train
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

linear_model = LinearRegression()
linear_model.fit(X_train_encoded, y_train)

y1_pred = linear_model.predict(X_test_encoded)

print("MAE:", mean_absolute_error(y_test, y1_pred))
print("MSE:", mean_squared_error(y_test, y1_pred))
print("R² Score:", r2_score(y_test, y1_pred))

MAE: 13841.274386327697
MSE: 3147570628.38631
R² Score: 0.7033541094019171


In [26]:
# Applying log transformation to response variable Price
df_log = df.copy()
df_log['Log_Price'] = np.log1p(df['Price'])
df_log = df_log.drop('Price', axis=1)

print(df_log.head())

                Model          Brand  Year  Mileage Transmission  Log_Price
0       Nissan Almera         Nissan  2015   149490    Automatic   9.952325
1        Perodua Aruz        Perodua  2019    88806    Automatic  10.831727
2       Daihatsu Gran       Daihatsu  2021   104943       Manual  10.596660
3  Mercedes-Benz C250  Mercedes-Benz  2010   147362    Automatic  10.668979
4         Proton Iriz         Proton  2019    52965    Automatic  10.370393


In [27]:
# Linear Regression Model with Log_Price
# Train-test split
X_log = df_log.drop('Log_Price', axis=1)
y_log = df_log['Log_Price']
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, test_size=0.2, random_state=42)

# Fit and transform X_train
X_train_encoded_log = encoder.fit_transform(X_train_log)
X_test_encoded_log = encoder.transform(X_test_log)

linear_model_log = LinearRegression()
linear_model_log.fit(X_train_encoded_log, y_train_log)

ylog_pred = linear_model_log.predict(X_test_encoded_log)

print("MAE:", mean_absolute_error(y_test_log, ylog_pred))
print("MSE:", mean_squared_error(y_test_log, ylog_pred))
print("R² Score:", r2_score(y_test_log, ylog_pred))

MAE: 0.1644383224595992
MSE: 0.06105058714651848
R² Score: 0.8508852679683998


In [49]:
# Random Forest Model
forest_model = RandomForestRegressor(random_state=42)

forest_model.fit(X_train_encoded, y_train)

y2_pred = forest_model.predict(X_test_encoded)

print("MAE:", mean_absolute_error(y_test, y2_pred))
print("MSE:", mean_squared_error(y_test, y2_pred))
print("R² Score:", r2_score(y_test, y2_pred))

MAE: 8991.986355199586
MSE: 3443324330.1889095
R² Score: 0.6754805107993367


In [47]:
# Random Forest Model
rf_model = RandomForestRegressor(random_state=42)

param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4),
    'max_features': ['auto', 'sqrt', 'log2']
}

random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42)
random_search.fit(X_train_encoded, y_train)

print("Best parameters found: ", random_search.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_p

Best parameters found:  {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 291}


In [48]:
best_rf_model = random_search.best_estimator_
best_rf_model.fit(X_train_encoded, y_train)

y2_pred = best_rf_model.predict(X_test_encoded)

print("MAE:", mean_absolute_error(y_test, y2_pred))
print("MSE:", mean_squared_error(y_test, y2_pred))
print("R² Score:", r2_score(y_test, y2_pred))

MAE: 15455.700646520694
MSE: 5754049572.721938
R² Score: 0.45770393691825895


In [18]:
# XGBoost
# handle categorical variables
categorical_columns = ['Model', 'Brand', 'Transmission']

for col in categorical_columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

xg_model = XGBRegressor(random_state=42, enable_categorical=True)
xg_model.fit(X_train, y_train)

y3_pred = xg_model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y3_pred))
print("MSE:", mean_squared_error(y_test, y3_pred))
print("R² Score:", r2_score(y_test, y3_pred))

MAE: 79284.34681146461
MSE: 22191567506.963882
R² Score: -1.0914661884307861


In [38]:
# finding best alpha score
# Ridge
ridge = Ridge()
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}  # Range of alpha values to test
grid_search = GridSearchCV(ridge, param_grid, cv=5)
grid_search.fit(X_train_encoded, y_train)

# Get the best parameter and score
print("Best alpha for Ridge:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best alpha for Ridge: {'alpha': 1}
Best cross-validation score: 0.8242080707258813


In [42]:
# finding best alpha score
# Lasso
lasso = Lasso()
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}  # Range of alpha values to test
grid_search = GridSearchCV(lasso, param_grid, cv=5)
grid_search.fit(X_train_encoded, y_train)

# Get the best parameter and score
print("Best alpha for Lasso:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 242604276012.4101, tolerance: 1257036534.6318822
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 411030943499.4887, tolerance: 1713171695.1171193
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 378202241209.9529, tolerance: 1467168156.32072
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:658: Converg

Best alpha for Lasso: {'alpha': 10}
Best cross-validation score: 0.7811740985704863


In [40]:
# Resolve overfitting through regularisation
# Lasso (L1 regularization)
lasso_model = Lasso(alpha=10)
lasso_model.fit(X_train_encoded, y_train)
y_pred_lasso = lasso_model.predict(X_test_encoded)

# Ridge (L2 regularization)
ridge_model = Ridge(alpha=1)
ridge_model.fit(X_train_encoded, y_train)
y_pred_ridge = ridge_model.predict(X_test_encoded)

# Evaluate models
print("Lasso Model RMSE:", mean_absolute_error(y_test, y_pred_lasso))
print("Lasso Model MSE:", mean_squared_error(y_test, y_pred_lasso))
print("Lasso Model R² Score:", r2_score(y_test, y_pred_lasso))

print("Ridge Model RMSE:", mean_absolute_error(y_test, y_pred_ridge))
print("Ridge Model MSE:", mean_squared_error(y_test, y_pred_ridge))
print("Ridge Model R² Score:", r2_score(y_test, y_pred_ridge))

Lasso Model RMSE: 13795.460529932627
Lasso Model MSE: 3157237183.824644
Lasso Model R² Score: 0.7024430753742262
Ridge Model RMSE: 14605.380539404277
Ridge Model MSE: 3991734200.872499
Ridge Model R² Score: 0.6237950829857224


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32737164364.497314, tolerance: 1923840442.9539526
  model = cd_fast.sparse_enet_coordinate_descent(


The logistic regression model will be choosen due to its highest R2 score.

In [51]:
# export chosen model
joblib.dump(linear_model_log, 'model.pkl')

['model.pkl']